In [ ]:
from gesri.alg.terrain import dem_from_tin

ELEVATION  = r'F:\fireloc\tst_miranda\elv\altimetria.shp'
ELEV_FIELD = 'ELEVATION'
BOUND_TIN  = r'F:\fireloc\tst_miranda\lmt\lmt_extra.shp'
BOUND_MDT  = r'F:\fireloc\tst_miranda\lmt\lmt.shp'
WWW        = r'F:\fireloc\tst_miranda\elv\mdt_tmp'
OUT_MDT    = r'F:\fireloc\tst_miranda\elv\mdt.tif'
EPSG       = 3763

dem_from_tin(
    ELEVATION, ELEV_FIELD, BOUND_TIN, BOUND_MDT,
    10, WWW, OUT_MDT, hidrology=None
)

In [ ]:
"""
Produce a MDT for each cell in vec frid file
"""

alti = r'C:\gwork\fireloc\datasets\mdt_m888\alti_pt25k_3763.shp'
ref  = r'C:\gwork\fireloc\datasets\ref\pt_ref_grid.shp'
outFld = r'C:\gwork\fireloc\datasets\mdt_m888'
cell_id = 'cellid'

cell_width = 3000
cell_height= 3000

import os; import datetime as dt
from glass3.fm          import tbl_to_obj
from glass3.g.gop.prox  import df_buffer_extent
from glass3.gt.prop.prj import get_epsg_shp
from glass3.gt.to.shp import coords_to_boundary
from glass3.pyt.oss import create_folder
from glass3.pyt.oss import fld_exists

from gesri.df.gop.ovlay import clip

epsg = 3763

grid_df = tbl_to_obj(ref)

grid_df = df_buffer_extent(grid_df, epsg, cell_width, mantainOriginalGeom=True)

In [ ]:
def clip_r(row):
    fld_path = os.path.join(outFld, 'data_' + str(row[cell_id]))
    
    isFld = fld_exists(fld_path)
    if not isFld:
        fld = create_folder(fld_path)
    else:
        continue
    
    # Get Lmt Shape
    left, bottom, right, top = row.old_geom.bounds
    lmt = coords_to_boundary((left, top), (right, bottom), epsg, os.path.join(
        fld, 'lmt_{}.shp'.format(str(row[cell_id]))
    ))
    
    # Get Extra Lmt
    left, bottom, right, top = row.geometry.bounds
    extra_lmt = coords_to_boundary((left, top), (right, bottom), epsg, os.path.join(
        fld, 'extra_lmt_{}.shp'.format(str(row[cell_id]))
    ))
    
    # Clip Altimetry
    alti_clp = clip(alti, extra_lmt, os.path.join(
        fld, 'alti_{}.shp'.format(str(row[cell_id]))
    ))
    
    return row

time_a = dt.datetime.now().replace(microsecond=0)
grid_df.apply(lambda x: clip_r(x), axis=1)
time_b = dt.datetime.now().replace(microsecond=0)
print(time_b - time_a)